In [23]:
# Projet d'AIKD
# XU Duo, SU Hao

# PART 1

In [24]:
# get 30 news
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key='888d5de05649432887e6499926707525')

all_articles = newsapi.get_everything(
                                      
                                      from_param='2018-11-15',
                                      to='2018-11-20',
                                      language='en',
                                      domains='bbc.com, bbc.co.uk '
                                      )
num_news = len(all_articles['articles'])
print("get {} news".format(num_news))

get 20 news


In [25]:
test_news_X = []
test_news_date = []
for i in range(20):
    test_news_X.append(all_articles['articles'][i]['content'])
    test_news_date.append(all_articles['articles'][i]['publishedAt'])

In [26]:
# get train dataset
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
print(twenty_train.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [27]:
# stop_word process
stop_words_I = []
with open('/Users/haosu/Desktop/停词.txt') as f:
    line = f.readline()
    while line:
        stop_words_I.append(line)
        line = f.readline()


In [28]:
#NLTK
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
def stop_word_process(text):
    stop_words = set(stopwords.words('english'))
    dataBank = []
    #Separate words
    for article in text:
        dataBank.append(word_tokenize(article))
    stop_words.add("I")
    stop_words.add(",")
    stop_words.add("(")
    stop_words.add(")")
    stop_words.add("-")
    stop_words.add("_")
    stop_words.add(".")
    stop_words.add(":")
    stop_words.add("A")
    stop_words.add("<")
    stop_words.add(">")
    stop_words.add("n't")
    stop_words.add("'s")
    stop_words.add("?")
    stop_words.add("!")
    stop_words.add("--")
    stop_words.add("chars")
    for i in range(len(stop_words_I)):
        stop_words.add(stop_words_I[i])
    # NLTK process
    twenty_train_split = []
    for i in range(len(dataBank)):
        filtered_sentence = []
        for word in dataBank[i]:
            if word not in stop_words:
                filtered_sentence.append(word)
        twenty_train_split.append(filtered_sentence)
    #twenty_train_split 是去除停词后的文本
    su = []
    for i in range(len(twenty_train_split)):
        su.append(" ".join(twenty_train_split[i]))
    return su

In [29]:
twenty_train_filtered = stop_word_process(twenty_train.data)

In [30]:
def removeNULL(text):
    # remove the null values
    for i in range(len(text)):
        if test_news_X[i] == None :
            #test_news_X.remove(text[i])
            test_news_X.pop(i)
            test_news_date.pop(i)
    return(text)

In [33]:
for i in range(len(test_news_X)):
    if isinstance(test_news_X[i],str):
        print("OK")  
    else: 
        print("NoneType")
        test_news_X = removeNULL(test_news_X)
thirty_test_filtered = stop_word_process(test_news_X)

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK


In [34]:
# train
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train_filtered)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(twenty_train_filtered, twenty_train.target)

from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train_filtered, twenty_train.target)
print(gs_clf.best_score_)
gs_clf.best_params_

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.9049849743680396


{'clf__alpha': 0.01, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}

In [35]:
# predict
X_new_counts = count_vect.transform(thirty_test_filtered)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = gs_clf.predict(thirty_test_filtered)
predict_result = []
for i in predicted:
    print(twenty_train.target_names[i])
    predict_result.append(twenty_train.target_names[i])
print("Best accuracy：%r" % (gs_clf.best_score_))

talk.politics.guns
talk.politics.guns
rec.sport.hockey
rec.motorcycles
rec.sport.hockey
sci.space
rec.motorcycles
talk.politics.misc
sci.med
sci.med
rec.sport.hockey
rec.sport.hockey
rec.sport.hockey
talk.politics.misc
rec.sport.hockey
talk.politics.guns
Best accuracy：0.9049849743680396


In [36]:
#test_news_X
#test_news_date
#predict_result
print(len(test_news_X))
print(len(test_news_date))
print(len(predict_result))
import pandas as pd
dataframe = pd.DataFrame({'content':test_news_X,'date':test_news_date,'label':predict_result})
dataframe.to_csv("/Users/haosu/Desktop/aikd.csv",index=False,sep=',')
#data = pd.read_csv('test.csv')

16
16
16


In [37]:
# KD
import uuid
print(uuid.uuid1())
'''
list1 -> ID
list3 -> label
list5 -> time
'''

e6d7e960-f7e7-11e8-8ace-acde48001122


'\nlist1 -> ID\nlist3 -> label\nlist5 -> time\n'

In [39]:
import rdflib

def create():
    g = rdflib.Graph()
    belongs_class_of = rdflib.URIRef('http://www.example.org/belongs_class_of')
    published_at = rdflib.URIRef('http://www.example.org/published_at')
    
    list1=[]
    list2=[]
    list3=[]   
    
    for i in range(len(test_news_X)):
        list1.append(rdflib.URIRef('http://www.example.org/'+"article{}".format(i+1)))
    print(list1)
   
    for i in predict_result:
       
        list2.append(rdflib.URIRef('http://www.example.org/'+i))
            
    for i in test_news_date:
        list3.append(rdflib.URIRef('http://www.example.org/'+i))

    for i in range(len(list1)):
         g.add((list1[i],belongs_class_of,list2[i]))
    for i in range(len(list1)):       
         g.add((list1[i],published_at,list3[i]))
    g.serialize("graph.rdf")

In [40]:
def query():
    g = rdflib.Graph()
    g.parse("graph.rdf", format="xml")
 
    q = "select ?article  where {?article <http://www.example.org/belongs_class_of>  <http://www.example.org/rec.sport.hockey> }"
    x = g.query(q)
    t = list(x) 
    print(len(t),'results found')
    
    for i in range(len(t)):
        print("Result {} of {} ".format(i+1,len(t)))
        print(t[i])
    # <?,b,?>
    
if __name__ == "__main__":
    create()
    query()

[rdflib.term.URIRef('http://www.example.org/article1'), rdflib.term.URIRef('http://www.example.org/article2'), rdflib.term.URIRef('http://www.example.org/article3'), rdflib.term.URIRef('http://www.example.org/article4'), rdflib.term.URIRef('http://www.example.org/article5'), rdflib.term.URIRef('http://www.example.org/article6'), rdflib.term.URIRef('http://www.example.org/article7'), rdflib.term.URIRef('http://www.example.org/article8'), rdflib.term.URIRef('http://www.example.org/article9'), rdflib.term.URIRef('http://www.example.org/article10'), rdflib.term.URIRef('http://www.example.org/article11'), rdflib.term.URIRef('http://www.example.org/article12'), rdflib.term.URIRef('http://www.example.org/article13'), rdflib.term.URIRef('http://www.example.org/article14'), rdflib.term.URIRef('http://www.example.org/article15'), rdflib.term.URIRef('http://www.example.org/article16')]
6 results found
Result 1 of 6 
(rdflib.term.URIRef('http://www.example.org/article15'),)
Result 2 of 6 
(rdflib.